<a href="https://colab.research.google.com/github/unofficial-Jona/ACML/blob/main/bounding_box_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# create the dataset

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Input
from tensorflow.keras.applications import VGG16
from keras.models import Sequential, Model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import patches
import os

In [2]:
pre_string = 'drive/MyDrive/hippo data/'
X_valid = np.load(pre_string + 'valid.npz')['x']/255
y_valid = np.load(pre_string + 'valid.npz')['y']
X_test = np.load(pre_string + 'test.npz')['x']/255
y_test = np.load(pre_string + 'test.npz')['y']
X_train = np.load(pre_string + 'train.npz')['x']/255
y_train = np.load(pre_string + 'train.npz')['y']

In [3]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
valid_data = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [4]:
batch_size = 256
shuffle_buffer_size = 100

train_data = train_data.shuffle(shuffle_buffer_size).batch(batch_size)
valid_data = valid_data.shuffle(shuffle_buffer_size).batch(batch_size)
test_data = test_data.batch(batch_size)

In [17]:
model = Sequential([
    Input((400,150,1)),
    Conv2D(8, kernel_size=(100,100), padding='same', activation = 'relu'),
    MaxPool2D((4,4)),
    Conv2D(12, kernel_size=(30,30), padding='same', activation = 'relu'),
    MaxPool2D((4,4)),
    Conv2D(32, kernel_size=(5,5), padding='same', activation = 'relu'),
    MaxPool2D((2,2)),
    Conv2D(64, kernel_size=(5,5), padding='same', activation = 'relu'),
    MaxPool2D((2,2)),
    Flatten(),
    # Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Dense(4, activation='sigmoid')
])

opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer = opt, loss='mse', metrics=['cosine_similarity', 'mse', 'mae'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 400, 150, 8)       80008     
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 100, 37, 8)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 100, 37, 12)       86412     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 25, 9, 12)        0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 25, 9, 32)         9632      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 12, 4, 32)       

In [18]:
model.fit(train_data, validation_data=valid_data, epochs=5, verbose=1)
model.evaluate(test_data)
predictions = model.predict(test_data)
predictions

Epoch 1/5
16/16 [==============================] - 616s 33s/step - loss: 0.0418 - cosine_similarity: 0.9507 - mse: 0.0418 - mae: 0.1605 - val_loss: 0.0393 - val_cosine_similarity: 0.9519 - val_mse: 0.0393 - val_mae: 0.1627
Epoch 2/5
12/16 [=====================>........] - ETA: 1:48 - loss: 0.0371 - cosine_similarity: 0.9585 - mse: 0.0371 - mae: 0.1568

KeyboardInterrupt: ignored

In [12]:
y_test

array([[0.46095309, 0.48923655, 0.57638893, 0.63238212],
       [0.26761973, 0.26261787, 0.38305557, 0.40576344],
       [0.45651204, 0.365714  , 0.63230911, 0.51528641],
       ...,
       [0.24682355, 0.45857563, 0.32973043, 0.59888229],
       [0.37144216, 0.52596863, 0.50301509, 0.66660316],
       [0.65539027, 0.40971764, 0.7895871 , 0.55446682]])

In [ ]:
model.save('drive/MyDrive/hippo data/model/first_try.h5')

In [ ]:
model = keras.models.load_model('drive/MyDrive/hippo data/model/first_try.h5')

In [5]:
import optuna

In [ ]:
def objective(trial):
  model = Sequential()

  model.add(Input((400,150,1)))

  model.add(
    Conv2D(
      filters = trial.suggest_categorical('filters', [8,16]),
      kernel_size = trial.suggest_categorical('kernel_size', [75,100]),
      activation = trial.suggest_categorical('activation', ['relu', 'linear']),
      padding='same'))
  
  model.add(MaxPool2D(
      pool_size = trial.suggest_categorical('pool_size', [2,4])))
  
  model.add(
    Conv2D(
      filters = trial.suggest_categorical('filters_1', [16, 32]),
      kernel_size = trial.suggest_categorical('kernel_size_1', [30,75]),
      activation = trial.suggest_categorical('activation_1', ['relu', 'linear']),
      padding='same'))
  
  model.add(MaxPool2D(
      pool_size = trial.suggest_categorical('pool_size_1', [2,4])))
  
  model.add(Flatten())

  model.add(Dense(
      units = trial.suggest_categorical('units', [64,128]),
      activation = trial.suggest_categorical('activation_2', ['relu', 'linear'])))

  model.add(Dense(
      units = trial.suggest_categorical('units_1', [32,64]),
      activation = trial.suggest_categorical('activation_3', ['relu', 'linear'])))

  model.add(Dense(
      units = trial.suggest_categorical('units_2', [8,16]),
      activation = trial.suggest_categorical('activation_3', ['relu', 'linear'])))

  model.add(Dense(
    units = 4,
    activation = 'sigmoid'))
  
  lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
  opt = tf.keras.optimizers.Adam(learning_rate=lr)

  model.compile(loss = 'mae', optimizer=opt, metrics=['mse', 'cosine_similarity', 'mae'])

  model.fit(train_data, validation_data=valid_data, epochs=5, verbose=1)
  mse = model.evaluate(test_data, verbose=0)
  return mse[0]

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=15)

[I 2022-01-03 18:25:21,723] A new study created in memory with name: no-name-ded7e6ef-bf92-46a2-8555-c8a3070831d1


Epoch 1/5
16/16 [==============================] - 754s 37s/step - loss: 0.2743 - mse: 0.2743 - cosine_similarity: 0.0633 - mae: 0.4793 - val_loss: 0.2858 - val_mse: 0.2858 - val_cosine_similarity: 0.0000e+00 - val_mae: 0.4948
Epoch 2/5
16/16 [==============================] - 458s 29s/step - loss: 0.2907 - mse: 0.2907 - cosine_similarity: 0.0000e+00 - mae: 0.5042 - val_loss: 0.2858 - val_mse: 0.2858 - val_cosine_similarity: 0.0000e+00 - val_mae: 0.4948
Epoch 3/5
16/16 [==============================] - 457s 29s/step - loss: 0.2907 - mse: 0.2907 - cosine_similarity: 0.0000e+00 - mae: 0.5042 - val_loss: 0.2858 - val_mse: 0.2858 - val_cosine_similarity: 0.0000e+00 - val_mae: 0.4948
Epoch 4/5
16/16 [==============================] - 453s 28s/step - loss: 0.2907 - mse: 0.2907 - cosine_similarity: 0.0000e+00 - mae: 0.5042 - val_loss: 0.2858 - val_mse: 0.2858 - val_cosine_similarity: 0.0000e+00 - val_mae: 0.4948
Epoch 5/5
16/16 [==============================] - 456s 29s/step - loss: 0.2907 

[I 2022-01-03 19:13:34,793] Trial 0 finished with value: 0.29401442408561707 and parameters: {'filters': 8, 'kernel_size': 100, 'activation': 'linear', 'pool_size': 2, 'filters_1': 16, 'kernel_size_1': 30, 'activation_1': 'relu', 'pool_size_1': 2, 'units': 64, 'activation_2': 'linear', 'units_1': 32, 'activation_3': 'linear', 'units_2': 16, 'lr': 0.0013498207426905352}. Best is trial 0 with value: 0.29401442408561707.


Epoch 1/5
16/16 [==============================] - 502s 30s/step - loss: 0.0324 - mse: 0.0324 - cosine_similarity: 0.9639 - mae: 0.1466 - val_loss: 0.0237 - val_mse: 0.0237 - val_cosine_similarity: 0.9764 - val_mae: 0.1304
Epoch 2/5
16/16 [==============================] - 471s 30s/step - loss: 0.0211 - mse: 0.0211 - cosine_similarity: 0.9776 - mae: 0.1211 - val_loss: 0.0223 - val_mse: 0.0223 - val_cosine_similarity: 0.9769 - val_mae: 0.1262
Epoch 3/5
16/16 [==============================] - 472s 30s/step - loss: 0.0205 - mse: 0.0205 - cosine_similarity: 0.9771 - mae: 0.1211 - val_loss: 0.0204 - val_mse: 0.0204 - val_cosine_similarity: 0.9782 - val_mae: 0.1193
Epoch 4/5
16/16 [==============================] - 472s 30s/step - loss: 0.0183 - mse: 0.0183 - cosine_similarity: 0.9801 - mae: 0.1134 - val_loss: 0.0195 - val_mse: 0.0195 - val_cosine_similarity: 0.9798 - val_mae: 0.1179
Epoch 5/5
11/16 [===================>..........] - ETA: 2:21 - loss: 0.0175 - mse: 0.0175 - cosine_similarit